In [ ]:
import numpy as np  
from PIL import Image  # for creating visual of our env
import cv2  # for showing our visual live
import matplotlib.pyplot as plt  # for results showing
from matplotlib import style  
import time  # using this to keep track of our saved Q-Tables.

style.use("ggplot")  # setting style
SIZE = 10
N_EPISODES = 25000
MOVE_PENALTY = 1 
Offloading_PENALTY = 300  # If the UE is not in need of the offloading operation , then offloading in this case is a waste of energy!
Offloading_REWARD = 25  # If the UE needs to offload a part of the computation task to the UAV server.
epsilon = 0.5  # Epsilon Greedy Strategy : To balance between exploration and exploitation
EPS_DECAY = 0.9999  # Every episode will be epsilon*EPS_DECAY : epsilon will tend to 0 which means we will be more eploitating the enviroment each episode
SHOW_EVERY = 1000  # how often to play through env visually.
LEARNING_RATE = 0.1 #how much we are taking into consederation the old q_values for the new value expressed by the bellman equation. 
DISCOUNT = 0.95 #how to take into consederation all the sequantial rewards in a prior range. 

d = {1: (255, 175, 0),  # blueish color ( the drone )
     2: (0, 255, 0),  # green#(UE in need of offloading)
     3: (0, 0, 255)}  # red( UE with no need of the offloading operation)
UAV= 1  # UAV key in d
UE_in_need = 2 # UE in need of offloading key in d 
UE_no_need = 3 #UE with no need of the offloading operation key in d

class Edge_device : #defining an object of edge device which can be a UAV or a UE 
    def __init__(self): # Each device has 3 parameters that defines it : its location in the env (x,y) and the calculation capacity of its server 
        self.x = np.random.randint(0, SIZE)
        self.y = np.random.randint(0, SIZE)
        self.capacity = np.random.randint(0,256)  
    def __str__(self): #loating each device method
        return f"{self.x}, {self.y}"
    def __sub__(self, other): #calculating the distance between two devices
        return (self.x-other.x, self.y-other.y)
    def action(self, choice): # or action_space contains 5 differnet actions which are movinf left , right , up or down and offloading in case of need 
    
        if choice == 0:
            self.move(x=1, y=1)
        elif choice == 1:
            self.move(x=-1, y=-1)
        elif choice == 2:
            self.move(x=-1, y=1)
        elif choice == 3:
            self.move(x=1, y=-1)
        elif choice == 4 :
             self.offload()
    def move(self, x=False, y=False):

        # If no value for x, move randomly
        if not x:
            self.x += np.random.randint(-1, 2)
        else:
            self.x += x

        # If no value for y, move randomly
        if not y:
            self.y += np.random.randint(-1, 2)
        else:
            self.y += y


        # If we are out of bounds
        if self.x < 0:
            self.x = 0
        elif self.x > SIZE-1:
            self.x = SIZE-1
        if self.y < 0:
            self.y = 0
        elif self.y > SIZE-1:
            self.y = SIZE-1
    def offload( self , other ) : 
        self.capacity += other.capacity

q_table = {}
for i in range(-SIZE+1, SIZE):
        for ii in range(-SIZE+1, SIZE):
            for iii in range(-SIZE+1, SIZE):
                    for iiii in range(-SIZE+1, SIZE):
                        for iiiii in range(0,256): 
                            for iiiiii in range(0,256):
                                q_table[((i, ii), (iii, iiii), (iiiii,iiiiii))] = [np.random.uniform(-5, 0) for i in range(5)]

In [ ]:
print(q_table[((-9, -2), (3, 9))])

In [ ]:
episode_rewards = [] # we gather all the reward from each episod in an list to add them to the whold rwards list for results showing

for episode in range(N_EPISODES):
    UAV = Edge_device()
    UE1 = Edge_device()
    UE2 = Edge_device()
    if episode % SHOW_EVERY == 0:
        print(f"on #{episode}, epsilon is {epsilon}")
        print(f"{SHOW_EVERY} ep mean: {np.mean(episode_rewards[-SHOW_EVERY:])}")
        show = True
    else:
        show = False
    episode_reward = 0
    for i in range(200): # 200 is the number of stups we chosed to make in a single episode
        obs = (UAV-UE1, UAV-UE2)
        
        if np.random.random() > epsilon: #the epsilon greedy strategy we talked about 
            # Exploitation
            action = np.argmax(q_table[obs])
        else:
            action = np.random.randint(0, 5)
        
        UAV.action(action)
        if ( UAV.x == UE1.x and UAV.y == UE1.y  and  UE1.capacity < UAV.capacity) :
            reward = - Offloading_PENALTY
        elif  ( UAV.x == UE2.x and UAV.y == UE2.y and UE2.capacity < UAV.capacity) :
            reward = - Offloading_PENALTY
        elif ( UAV.x == UE1.x and UAV.y == UE1.y and  UE1.capacity > UAV.capacity) : 
            reward = Offloading_REWARD
        elif ( UAV.x == UE2.x and UAV.y == UE2.y  ) and ( UE2.capacity > UAV.capacity) : 
            reward = Offloading_REWARD
        else : 
            reward = - MOVE_PENALTY 
        new_obs = (UAV-UE1, UAV-UE2 ,  (UAV.capacity - UE1.capacity , UAV.capacity - UE2.capacity) ) # new observation
        future_q = np.max(q_table[new_obs])  # max Q value for this new obs
        current_q = q_table[obs][action]  # current Q for our chosen action
        if reward == Offloading_REWARD:
            new_q = Offloading_REWARD
        else:
            new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * future_q)
        if show:
            env = np.zeros((SIZE, SIZE, 3), dtype=np.uint8)  # starts an rbg of our size
            if UE1.capacity < UAV.capcity : 
                 env[UE1.x][UE1.y] = d[UE_no_need_N]
            elif UE2.capacity < UAV.capcity : 
                 env[UE2.x][UE2.y] = d[UE_no_need_N]
            elif UE1.capacity > UAV.capcity : 
                 env[UE1.x][UE1.y] = d[UE_in_need]
            elif UE2.capacity > UAV.capcity : 
                 env[UE2.x][UE1.y] = d[UE_in_need]
            env[UAV.x][UAV.y] = d[UAV_N]  # sets the player tile to blue
            img = Image.fromarray(env, 'RGB')  # reading to rgb. Apparently. Even tho color definitions are bgr. ???
            img = img.resize((300, 300))  # resizing so we can see our agent in all its glory.
            cv2.imshow("image", np.array(img))  # show it!
            if reward == Offload_REWARD or reward == -Offloading_PENALTY:  # crummy code to hang at the end if we reach abrupt end for good reasons or not.
                if cv2.waitKey(500) & 0xFF == ord('q'):
                    break
            else:
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            episode_reward += reward
            if reward == Offloading_REWARD or reward == -Offloading_PENALTY:
                break
            episode_rewards.append(episode_reward)
    epsilon *= EPS_DECAY
    moving_avg = np.convolve(episode_rewards, np.ones((SHOW_EVERY,))/SHOW_EVERY, mode='valid')

plt.plot([i for i in range(len(moving_avg))], moving_avg)
plt.ylabel(f"Reward {SHOW_EVERY}ma")
plt.xlabel("episode #")
plt.show()